<a href="https://colab.research.google.com/github/RK900/cs182_project/blob/main/nlp_discriminator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Stuff

In [2]:
from google.colab import drive
drive.mount('/content/drive')
root_folder = "/content/drive/My Drive/cs182_project/"

Mounted at /content/drive


In [3]:
import sys

In [4]:
sys.path.append(root_folder)

In [ ]:
!pip install transformers

# Code

In [6]:
import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt
% matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup
from data_parsing import load_dataset, load_gen_dataset

## Load Data and Models

In [7]:
data_path = os.path.join(root_folder, "yelp_review_training_dataset.jsonl")
aug_data = load_gen_dataset(os.path.join(root_folder, "new_data.json")) + load_gen_dataset(os.path.join(root_folder, "new_data2.json")) + load_gen_dataset(os.path.join(root_folder, "new_data3.json"))

In [8]:
data = load_dataset(data_path)

In [9]:
output_dir = os.path.join(root_folder, "./model_save_gpt_234/")

In [10]:
config = GPT2Config.from_pretrained(output_dir)

In [ ]:
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [13]:
device = torch.device('cuda')

## Generate Sample Predictions

In [ ]:
model.eval()

prompt = "<|startoftext|> OMG I absolutely loved"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)
model = model.to(device)

print(generated)

sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=3
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

## Build PyTorch Transformer Discriminator

In [15]:
disriminator = torch.nn.Transformer(d_model=256)